In [1]:
import pandas as pd
import numpy as np

from stump import *

In [2]:
#converting input to the required format

from sklearn.impute import SimpleImputer

train = pd.read_csv('train_classif.csv')
test = pd.read_csv('test_classif.csv')

relevant_features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']

imputer = SimpleImputer(strategy='most_frequent')
train[relevant_features] = imputer.fit_transform(train[relevant_features])
test[relevant_features] = imputer.transform(test[relevant_features])

#encoding
train['Sex'] = train['Sex'].map({'male':0, 'female':1})
test['Sex'] = test['Sex'].map({'male':0, 'female':1})
train['Embarked'] = train['Embarked'].map({'S':0,'C':1,'Q':2})
test['Embarked'] = test['Embarked'].map({'S':0,'C':1,'Q':2})

X = train[relevant_features].to_numpy()
X_test = test[relevant_features].to_numpy()
Y = train['Survived'].to_numpy()

In [3]:
class adaboost_for_classif:
    
    def __init__(self):
        self.stump_list = []
        
    def add_stump(self,X,Y,weights):
        
        o = stump()
        weights = o.best_stump(X,Y,weights,'classif')
        self.stump_list.append(o)
        
        return weights
                
    def build(self, X, Y, n_iter):
        
        # giving equal weight to all datapoints
        weights = [1/X.shape[0] for i in range(X.shape[0])]
        
        for i in range(n_iter):
            weights = self.add_stump(X,Y,weights)
            

In [4]:
print(X.shape)

o = stump()
weights = [1/X.shape[0] for i in range(X.shape[0])]
o.best_stump(X,Y,weights,'classif')
print(o)

# test['Survived'] = o.predict(X_test)
# submissions = test[['PassengerId', 'Survived']]

# submissions.to_csv('submission.csv', index=False )

(891, 7)
<class 'list'>


In [6]:
print(o.left,o.right,o.stump_weight,o.threshold,o.feature)
print(X)

<stump.leaf object at 0x00000187E7BCF160> <stump.leaf object at 0x00000187E7BCEFE0> 0.23664385201205487 0 1
[[3 0 22.0 ... 0 7.25 0]
 [1 1 38.0 ... 0 71.2833 1]
 [3 1 26.0 ... 0 7.925 0]
 ...
 [3 1 24.0 ... 2 23.45 0]
 [1 0 26.0 ... 0 30.0 1]
 [3 0 32.0 ... 0 7.75 2]]
